In [57]:
import numpy as np
import matplotlib.pyplot as plt
import random
import gym
from gym import spaces

In [58]:
class UAV_env(gym.Env):
    def __init__(self):
        # UAV parameters
        self.m = 1.56  # Mass (kg)
        self.Jyy = 0.0576  # Moment of inertia (kg-m^2)
        self.CL = lambda a: (3.5*a) + 0.09  # Lift coefficient
        self.CD = lambda a: (0.2*a) + 0.016  # Drag coefficient
        self.CLdelta = 0.27  # Lift coefficient derivative
        self.Cm0 = -0.02  # Pitching moment coefficient at zero angle of attack
        self.Cmalpha = -0.57  # Pitching moment coefficient derivative
        self.Cmq = -1.4  # Pitching moment coefficient derivative
        self.Cmdelta = -0.32  # Pitching moment coefficient derivative

        # Initial state
        self.X0 = np.array([9.96, 0.87, 0, 0.0873, 50])  # [u, w, q, theta, h]
        self.U0 = np.array([1.0545, -0.2179])  # [T, delta]

        # Desired final state
        self.Xd = np.array([9.85, 1.74, 0, 0.1745, 62])

        # State and control constraints
        self.T_min = 0
        self.T_max = 4
        self.alpha_min = 0
        self.alpha_max = 0.2618
        self.Va_min = 5
        self.Va_max = 15
        self.delta_min = -0.4363
        self.delta_max = 0.1745

        # Simulation time
        self.dt = 0.02  # Sampling time (seconds)
        self.t_final = 10  # Final time (seconds)
        self.t = 0

        # Define action and observation spaces
        self.action_space = spaces.Box(low=np.array([self.T_min, self.delta_min]),
                                       high=np.array([self.T_max, self.delta_max]),
                                       dtype=np.float32)
        
        # self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(5,), dtype=np.float32)

        self.state = None

    def dynamics(self, X, U):
        u, w, q, theta, h = X       # States
        T, delta = U                # Controls

        Va = np.sqrt(u**2 + w**2)   # Airspeed (need to check this to keep within constraints)
        alpha = np.arctan(w / u)    # Angle of attack (need to check this to keep within constraints)

        CL = self.CL(alpha)         # Lift coefficient
        CD = self.CD(alpha)         # Drag coefficient

        udot = (-q*w) + (0.5*Va**2/self.m) * (CL*np.sin(alpha) - CD*np.cos(alpha) + self.CLdelta*np.sin(alpha)*delta) - (9.81*np.sin(theta)) + (T/self.m)
        wdot = (q*u) + (0.5*Va**2/self.m) * (-CL*np.cos(alpha) - CD*np.sin(alpha) - self.CLdelta*np.cos(alpha)*delta) + (9.81*np.cos(theta))
        qdot = (0.5*Va**2/self.Jyy) * (self.Cm0 + self.Cmalpha*alpha + (0.25*self.Cmq*q/Va) + self.Cmdelta*delta)
        thetadot = q
        hdot = (-u*np.sin(theta)) + (w*np.cos(theta))

        return np.array([udot, wdot, qdot, thetadot, hdot]) # Return state derivatives

    def step(self, action):
        action = np.clip(action, self.action_space.low, self.action_space.high)
        self.state = self.integrate(self.state, action) 
        u, w, _, _, _ = self.state
        Va = np.sqrt(u**2 + w**2)
        alpha = np.arctan(w / u)
        if Va < self.Va_min or Va > self.Va_max or alpha < self.alpha_min or alpha > self.alpha_max:
            reward = -np.inf  # Penalize out of bounds states
            done = True
        reward = -np.sum(np.abs(self.state - self.Xd)) # Negative sum of absolute state errors
        done = False
        if self.t >= 10:
            done = True
        info = {} # store additional metadata for debugging 
        return self.state, reward, done, info

    def reset(self):
        self.state = np.copy(self.X0)
        self.t = 0
        return self.state

    def integrate(self, X, U):
        X_dot = self.dynamics(X, U)
        X += self.dt * X_dot
        self.t += self.dt
        return X

    def render(self, mode='human'):
        # print(self.state)  
        pass

In [63]:
env = UAV_env()
state = env.reset()
num_episodes = 10
for _ in range(num_episodes):
    done = False
    score = 0
    while not done:
        if env.t == 0:
            action = env.U0
        else:
            # action = DQN(state)  # Use DQN in keras-rl2 to get our actual action policy
            action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode score: {score}")
env.close()

Episode score: -10376.619591643122
Episode score: -25.56321200652404
Episode score: -29.334448757309257
Episode score: -25.743912355925083
Episode score: -25.140450336038814
Episode score: -24.642498488287693
Episode score: -30.054335608523424
Episode score: -28.980152668376796
Episode score: -24.430714220787614
Episode score: -26.923249432302356
